In [1]:
import pickle
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

C:\Users\Yi\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
file = ['../datasets/train_data_jieba.txt', '../datasets/valid_data_jieba.txt', '../datasets/test_data_jieba.txt']
X_train = []
X_valid = []
X_test = []

with open('../datasets/train_data_jieba.txt', 'r', encoding='utf8') as handle:
    for line in handle.readlines():
        X_train.append(line.rstrip('\n'))
        
        
with open('../datasets/valid_data_jieba.txt', 'r', encoding='utf8') as handle:
    for line in handle.readlines():
        X_valid.append(line.rstrip('\n'))
        
        
with open('../datasets/test_data_jieba.txt', 'r', encoding='utf8') as handle:
    for line in handle.readlines():
        X_test.append(line.rstrip('\n'))

In [3]:
name = ['location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again']
train_data = pd.read_csv('../datasets/sentiment_analysis_trainingset.csv')
valid_data = pd.read_csv('../datasets/sentiment_analysis_validationset.csv')

y_train = []
y_valid = []
for N in name:
    Y_train = train_data[[N]]
    Y_train = np.array(Y_train)+2
    Y_train = to_categorical(Y_train, num_classes=4)
    y_train.append(Y_train)
    
    Y_valid = valid_data[[N]]
    Y_valid = np.array(Y_valid)+2
    Y_valid = to_categorical(Y_valid, num_classes=4)
    y_valid.append(Y_valid)

In [4]:
embedding_path = '../datasets/tencent_embedding.npy'
# 词典大小，embedding矩阵大小
tok2idx_path = '../datasets/vocab.pickle'

In [5]:
embedding_matrix = np.load(embedding_path)
with open(tok2idx_path, 'rb') as file:
    tok2idx = pickle.load(file)

In [6]:
def tokenize(sentence, tok2idx):
    return [tok2idx[word] for word in sentence.split(' ')]

maxlen = 1024
train = [tokenize(x, tok2idx) for x in X_train]
valid = [tokenize(x, tok2idx) for x in X_valid]
test = [tokenize(x, tok2idx) for x in X_test]
x_train = pad_sequences(train, maxlen=maxlen, padding='pre')
x_valid = pad_sequences(valid, maxlen=maxlen, padding='pre')
x_test = pad_sequences(test, maxlen=maxlen, padding='pre')

# Builde Model

In [9]:
from keras.layers import Embedding, Conv1D, MaxPooling1D, Input, GlobalAveragePooling1D, Dense
from keras import backend as K
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))
K.clear_session()

In [10]:
embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            trainable=False)

In [11]:
sentence_input = Input((maxlen,))

In [12]:
sentence_embed = embedding_layer(sentence_input)

In [13]:
conv1 = Conv1D(10, 3, activation='relu')(sentence_embed)
conv
conv2 = Conv1D(10, 3, activation='relu')(conv1)
conv3 = Conv1D(10, 3, activation='relu')(conv2)
flat = GlobalAveragePooling1D()(conv3)

out_list = []
for target in name:
    out = Dense(4, activation='softmax', name=target)(flat)
    out_list.append(out)

In [14]:
from keras import Model

In [15]:
model = Model(sentence_input, out_list)

In [16]:
model.compile('adam', 'categorical_crossentropy')

In [17]:
model.fit(x_train, y_train)

Epoch 1/1
105000/105000 [==============================] - 103s 985us/step - loss: 16.0326 - location_traffic_convenience_loss: 0.5892 - location_distance_from_business_district_loss: 0.5481 - location_easy_to_find_loss: 0.6991 - service_wait_time_loss: 0.4994 - service_waiters_attitude_loss: 1.1267 - service_parking_convenience_loss: 0.3103 - service_serving_speed_loss: 0.5924 - price_level_loss: 1.2207 - price_cost_effective_loss: 0.7309 - price_discount_loss: 0.9894 - environment_decoration_loss: 0.9187 - environment_noise_loss: 0.8045 - environment_space_loss: 0.9617 - environment_cleaness_loss: 0.8869 - dish_portion_loss: 1.1189 - dish_taste_loss: 0.9363 - dish_look_loss: 0.7944 - dish_recommendation_loss: 0.6221 - others_overall_experience_loss: 0.8110 - others_willing_to_consume_again_loss: 0.8717
